# Projeto Google Sheets

### Objetivo do projeto:

- Utilizar os dados de uma planilha do Google Sheets (https://docs.google.com/spreadsheets/d/1DYO-AjAZFki21a1ds-E4AKVCPIUoMKA98wneGYCeluk/edit?usp=sharing);
- Automatizar a atualização do status dessa planilha;
- Automatizar o envio de e-mail para as pessoas em que o status for atualizado.

### Contexto da planilha:

- Exemplo de dados cadastrados em alguma plataforma de vendas online;
- Onde a coluna de "Problemas" seria organizada por tipos de problemas para facilitar a automação do status.

### Para utilizar o Projeto

- Copie e atualize os e-mails da planilha de exemplo para e-mails que você use para realizar o teste;
- Utilize sua conta do gmail na função abaixo para poder enviar os e-mails;
- Opcional: Mude o nome no final do texto do e-mail.

### Crie um Projeto no Google Cloud utilizando a API do Google Sheets

- API Google Sheets: https://developers.google.com/sheets/api/guides/concepts
- Criar projeto Google Cloud: https://developers.google.com/workspace/guides/create-project


- Tenha os arquivos 'credentials.json' e 'token.json'.


### Instaladores

- pip install yagmail
- pip install google-api-python-client
- pip install google_auth_oauthlib

### Função de envio de email

In [15]:
import yagmail

usuario = yagmail.SMTP(user='seuemail@gmail.com', password='suasenha')

def enviar_email(dados, mensagem):
    corpo_email = [
        'Olá {}, tudo bem?'.format(lista[0]),
        '{}'.format(mensagem),
        'Att.,',
        'Seu Nome'
    ]
    corpo_email = '\n'.join(corpo_email)

    usuario.send(to='{}'.format(lista[1]), subject='Status da sua compra: {}'.format(lista[2]), contents=corpo_email)

### Função de adicionar status no Google Sheets

In [11]:
def adiciona_status(local):
    result = service.spreadsheets().values().update(
                spreadsheetId=SAMPLE_SPREADSHEET_ID,
                range=f"Página1!D{linha}",
                valueInputOption="USER_ENTERED", body=body).execute()

### Fazendo conexão com a planilha do Google Sheets

In [7]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# ID da planilha
SAMPLE_SPREADSHEET_ID = '1DYO-AjAZFki21a1ds-E4AKVCPIUoMKA98wneGYCeluk'
# Página e os dados que deve usar
SAMPLE_RANGE_NAME = 'Página1!A2:D9'

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])

if not values:
    print('No data found.')
else:
    print("Conexão Bem sucedida")

Conexão Bem sucedida


### Preenchendo Status da planilha e enviando e-mails

In [13]:
# Passo 1: Ler o intervalo de células
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])

# Passo 2: Verificar se o status está preenchido
linha = 2
for lista in values:
    # Passo 3: Se o status não estiver preenchido, verificar o 'Problema'
    if len(lista) < 4:
        # Passo 4: Enviar Mensagem e registrar novo status
        if lista[2] == "Boleto Gerado":
            # Enviando e-mail 
            enviar_email(lista, 'O boleto de sua compra já foi gerado!')
            
            # preencher a coluna 'Status' com "Mensagem do Boleto enviada"
            values = [
                [
                    "Mensagem do Boleto enviada"
                ],
            ]
            body = {
                'values': values
            }
            adiciona_status(lista)


        elif lista[2] == "Comprou":
            # Enviando email 
            enviar_email(lista, 'Sua compra já foi finalizada!')
            
            # preencher a coluna 'Status' com "Compra Finalizada"
            values = [
                [
                    "Compra Finalizada"
                ],
            ]
            body = {
                'values': values
            }
            adiciona_status(lista)

        elif lista[2] == "Sem Saldo":
            # Enviando email 
            enviar_email(lista, 'Não foi possível efetuar sua compra: Sem Saldo!')
            
            # preencher a coluna 'Status' com "Mensagem Sem Saldo enviada"
            values = [
                [
                    "Mensagem Sem Saldo enviada"
                ],
            ]
            body = {
                'values': values
            }
            adiciona_status(lista)

    linha += 1